# Best ANN, No Transfer

In [38]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import pandas as pd
from skimage import io
from PIL import Image
import functools
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from keras.metrics import categorical_accuracy
os.chdir("C:/Users/jerem.DESKTOP-GGM6Q2I/Documents/UNH Data Analytics/NN")

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

#### Neural Network Structure

In [39]:
from keras import models
from keras import layers
from keras.layers import BatchNormalization, Flatten, Dense, LeakyReLU, Dropout, GaussianNoise
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras_gradient_noise import add_gradient_noise
from keras.initializers import glorot_uniform, glorot_normal, he_uniform, he_normal
set_random_seed(1724)
shape=64


model = models.Sequential()
model.add(Flatten(input_shape=(shape,shape,3)))
model.add(Dense(128, kernel_initializer="glorot_normal"))
model.add(LeakyReLU())
model.add(Dropout(.2))
model.add(BatchNormalization())
model.add(GaussianNoise(1))
model.add(Dense(128))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dense(38, activation='softmax'))
# model.add(Dense(4, activation='softmax'))

#### Compiling Network

In [40]:
noisy = add_gradient_noise(Adam)
model.compile(optimizer=noisy(),
# model.compile(optimizer="adam",
                loss='categorical_crossentropy',
                metrics=[categorical_accuracy])

#### Setup for Early Stopping

In [41]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

#### Train Model

In [2]:
batch_size = 128
train_dir = "C:/Project/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
test_dir = "C:/Project/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"
# train_dir = "C:/Project/train/train"
# test_dir = "C:/Project/valid/valid"
nb_train_samples = sum([len(files) for r, d, files in os.walk(train_dir)])
nb_test_samples = sum([len(files) for r, d, files in os.walk(test_dir)])
epochs = 50


train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=False, vertical_flip=False, zca_whitening=False, zca_epsilon=.2, width_shift_range=0, height_shift_range=0, rotation_range=0, brightness_range=(0,1), shear_range=0, zoom_range=0.0)
test_datagen = ImageDataGenerator(rescale=1./255) #don't add anything else to this one, only train datagen
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                   target_size=(shape,shape),
                                                   color_mode="rgb",
                                                   batch_size=batch_size,
                                                   class_mode="categorical",
                                                   shuffle=True)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(shape,shape),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical"
                                                    )
history = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size+1,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=nb_test_samples // batch_size+1,
                    #use_multiprocessing=True,
                    workers=4,
                    callbacks=callbacks
)

#### Evaluate Model

In [ ]:
from keras.models import load_model
model = load_model('best_model.h5', custom_objects={"NoisyAdam":noisy()})
# model = load_model('best_model.h5')
model.evaluate_generator(test_generator, steps=nb_test_samples // batch_size+1, verbose=0)


# Best ANN, Transfer

In [33]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import pandas as pd
from skimage import io
from PIL import Image
import functools
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from keras.metrics import categorical_accuracy
os.chdir("C:/Users/jerem.DESKTOP-GGM6Q2I/Documents/UNH Data Analytics/NN")

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

#### Neural Network Structure

In [34]:
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import MobileNet
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import BatchNormalization, Flatten, Dense, LeakyReLU, Dropout, GaussianNoise
from tensorflow import set_random_seed
from keras.optimizers import Adam
from keras_gradient_noise import add_gradient_noise
from keras.initializers import glorot_uniform, glorot_normal, he_uniform, he_normal
set_random_seed(1724)
shape=64

batch_size = 128
train_dir = "C:/Project/64/train"
test_dir = "C:/Project/64/valid"
# train_dir = "C:/Project/train/train"
# test_dir = "C:/Project/valid/valid"
nb_train_samples = sum([len(files) for r, d, files in os.walk(train_dir)])
nb_test_samples = sum([len(files) for r, d, files in os.walk(test_dir)])
epochs = 1

base_model=VGG16(weights="imagenet", include_top=False, input_shape=(shape,shape,3))
x= base_model.output
x = Flatten()(x)
x= (Dense(128, kernel_initializer="glorot_normal"))(x)
x = (LeakyReLU())(x)
x = (Dropout(.2))(x)
x = (BatchNormalization())(x)
x = (GaussianNoise(1))(x)
x = (Dense(128))(x)
x = (LeakyReLU())(x)
x = (BatchNormalization())(x)
pred = (Dense(38, activation='softmax'))(x)
model = Model(inputs=base_model.input, outputs=pred)





#### Compiling Network

In [35]:
noisy = add_gradient_noise(Adam)
model.compile(optimizer=noisy(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])


#### Setup for Early Stopping

In [36]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

#### Train Model

In [1]:
epochs=2
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=False, vertical_flip=False, zca_whitening=True, zca_epsilon=.2, width_shift_range=0.2, height_shift_range=0, rotation_range=0, brightness_range=(0,1), shear_range=0, zoom_range=0.0)
test_datagen = ImageDataGenerator(rescale=1./255) #don't add anything else to this one, only train datagen
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                   target_size=(shape,shape),
                                                   color_mode="rgb",
                                                   batch_size=batch_size,
                                                   class_mode="categorical",
                                                   shuffle=True)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(shape,shape),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical"
                                                    )
history = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size+1,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=nb_test_samples // batch_size+1,
                    workers=4,
                    callbacks=callbacks
)



#### Evaluate Model

In [ ]:
from keras.models import load_model
model = load_model('best_model.h5', custom_objects={"NoisyAdam":noisy()})
model.evaluate_generator(test_generator, steps=nb_test_samples // batch_size+1, verbose=0)


# Cnn Model #1

In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import pandas as pd
from skimage import io
from PIL import Image
import functools
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from keras.metrics import categorical_accuracy
os.chdir("C:/Users/jerem.DESKTOP-GGM6Q2I/Documents/UNH Data Analytics/NN")

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

Using TensorFlow backend.


#### Neural Network Structure

In [11]:
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import MobileNet
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import BatchNormalization, Flatten, Dense, LeakyReLU, Dropout, GaussianNoise
from tensorflow import set_random_seed
from keras.optimizers import Adam
from keras_gradient_noise import add_gradient_noise
from keras.initializers import glorot_uniform, glorot_normal, he_uniform, he_normal
from keras.layers.convolutional import Conv2D, MaxPooling2D
set_random_seed(1724)
shape=64

batch_size = 128   ###############################################################################
train_dir = "C:/Project/64/train" ###############################################################
test_dir = "C:/Project/64/valid"  ##################################################################
# train_dir = "C:/Project/train/train"
# test_dir = "C:/Project/valid/valid"
nb_train_samples = sum([len(files) for r, d, files in os.walk(train_dir)])
nb_test_samples = sum([len(files) for r, d, files in os.walk(test_dir)])
epochs = 150


################################################################################################
# base_model=VGG16(weights="imagenet", include_top=False, input_shape=(shape,shape,3))
# x= base_model.output
# x = Flatten()(x)
# x= (Dense(128, kernel_initializer="glorot_normal"))(x)
# x = (LeakyReLU())(x)
# x = (Dropout(.2))(x)
# x = (BatchNormalization())(x)
# x = (GaussianNoise(1))(x)
# x = (Dense(128))(x)
# x = (LeakyReLU())(x)
# x = (BatchNormalization())(x)
# pred = (Dense(38, activation='softmax'))(x)
# model = Model(inputs=base_model.input, outputs=pred)



# x = models.Sequential()
# x.add(Dense(128, kernel_initializer="glorot_normal", input_shape = (shape,shape,3)))
# x.add(Flatten())
# x.add(LeakyReLU())
# x.add(Dropout(.2))
# x.add(BatchNormalization())
# x.add(GaussianNoise(1))
# x.add(Dense(128))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Dense(4, activation='softmax')) ################################################################

x = models.Sequential()

x.add(Conv2D(16, kernel_size = (3,3), strides = (1,1), padding = 'same', input_shape = (shape,shape,3)))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))
# x.add(Dropout(.2))

x.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
# x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))
# x.add(Dropout(.2))

# x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# # x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# # x.add(LeakyReLU())
# # x.add(BatchNormalization())
# x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))
# x.add(Dropout(.2))

x.add(Flatten())
x.add(Dense(64, kernel_initializer="glorot_normal"))
x.add(LeakyReLU())
x.add(BatchNormalization())
# x.add(Dropout(.4))
x.add(Dense(38, activation='softmax')) ################################################################

#### Compiling Network

In [12]:
noisy = add_gradient_noise(Adam)
x.compile(optimizer=noisy(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])


#### Setup for Early Stopping

In [13]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

#### Train Model

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=False, vertical_flip=False, zca_whitening=True, zca_epsilon=.2, width_shift_range=0.2, height_shift_range=0, rotation_range=0, brightness_range=(0,1), shear_range=0, zoom_range=0.0)
test_datagen = ImageDataGenerator(rescale=1./255) #don't add anything else to this one, only train datagen
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                   target_size=(shape,shape),
                                                   color_mode="rgb",
                                                   batch_size=batch_size,
                                                   class_mode="categorical",
                                                   shuffle=True)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(shape,shape),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical"
                                                    )
history = x.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size+1,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=nb_test_samples // batch_size+1,
                    workers=4,
                    callbacks=callbacks
)




#### Evaluate Model

In [32]:
from keras.models import load_model
model = load_model('best_model.h5', custom_objects={"NoisyAdam":noisy()})
model.evaluate_generator(test_generator, steps=nb_test_samples // batch_size+1, verbose=0)


# CNN Model #2

In [16]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import pandas as pd
from skimage import io
from PIL import Image
import functools
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from keras.metrics import categorical_accuracy
os.chdir("C:/Users/jerem.DESKTOP-GGM6Q2I/Documents/UNH Data Analytics/NN")

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

#### Neural Network Structure

In [17]:
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import MobileNet
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import BatchNormalization, Flatten, Dense, LeakyReLU, Dropout, GaussianNoise
from tensorflow import set_random_seed
from keras.optimizers import Adam
from keras_gradient_noise import add_gradient_noise
from keras.initializers import glorot_uniform, glorot_normal, he_uniform, he_normal
from keras.layers.convolutional import Conv2D, MaxPooling2D
set_random_seed(1724)
shape=64

batch_size = 128   
train_dir = "C:/Project/64/train" 
test_dir = "C:/Project/64/valid"  
nb_train_samples = sum([len(files) for r, d, files in os.walk(train_dir)])
nb_test_samples = sum([len(files) for r, d, files in os.walk(test_dir)])
epochs = 150





x = models.Sequential()

x.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same', input_shape = (shape,shape,3)))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))
# x.add(Dropout(.2))

x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
# x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))
# x.add(Dropout(.2))

# x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# # x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# # x.add(LeakyReLU())
# # x.add(BatchNormalization())
# x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))
# x.add(Dropout(.2))

x.add(Flatten())
x.add(Dense(256, kernel_initializer="glorot_normal"))
x.add(LeakyReLU())
x.add(BatchNormalization())
# x.add(Dropout(.4))
x.add(Dense(38, activation='softmax')) 

#### Compiling Network

In [18]:
noisy = add_gradient_noise(Adam)
x.compile(optimizer=noisy(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])


#### Setup for Early Stopping

In [19]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

#### Train Model

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=False, vertical_flip=False, zca_whitening=True, zca_epsilon=.2, width_shift_range=0.2, height_shift_range=0, rotation_range=0, brightness_range=(0,1), shear_range=0, zoom_range=0.0)
test_datagen = ImageDataGenerator(rescale=1./255) #don't add anything else to this one, only train datagen
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                   target_size=(shape,shape),
                                                   color_mode="rgb",
                                                   batch_size=batch_size,
                                                   class_mode="categorical",
                                                   shuffle=True)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(shape,shape),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical"
                                                    )
history = x.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size+1,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=nb_test_samples // batch_size+1,
                    workers=4,
                    callbacks=callbacks
)




#### Evaluate Model

In [ ]:
from keras.models import load_model
model = load_model('best_model.h5', custom_objects={"NoisyAdam":noisy()})
model.evaluate_generator(test_generator, steps=nb_test_samples // batch_size+1, verbose=0)


# CNN Model #3

In [21]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import pandas as pd
from skimage import io
from PIL import Image
import functools
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from keras.metrics import categorical_accuracy
os.chdir("C:/Users/jerem.DESKTOP-GGM6Q2I/Documents/UNH Data Analytics/NN")

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

#### Neural Network Structure

In [26]:
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import MobileNet
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import BatchNormalization, Flatten, Dense, LeakyReLU, Dropout, GaussianNoise
from tensorflow import set_random_seed
from keras.optimizers import Adam
from keras_gradient_noise import add_gradient_noise
from keras.initializers import glorot_uniform, glorot_normal, he_uniform, he_normal
from keras.layers.convolutional import Conv2D, MaxPooling2D
set_random_seed(1724)
shape=64

batch_size = 128   
train_dir = "C:/Project/64/train" 
test_dir = "C:/Project/64/valid"  
nb_train_samples = sum([len(files) for r, d, files in os.walk(train_dir)])
nb_test_samples = sum([len(files) for r, d, files in os.walk(test_dir)])
epochs = 150




x = models.Sequential()

x.add(Conv2D(16, kernel_size = (3,3), strides = (1,1), padding = 'same', input_shape = (shape,shape,3)))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
# x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
# x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x.add(Flatten())
x.add(Dense(128, kernel_initializer="glorot_normal"))
x.add(LeakyReLU())
x.add(BatchNormalization())

x.add(Dense(38, activation='softmax')) 

#### Compiling Network

In [27]:
noisy = add_gradient_noise(Adam)
x.compile(optimizer=noisy(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])


#### Setup for Early Stopping

In [28]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

#### Train Model

In [5]:
# train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=False, vertical_flip=False, zca_whitening=True, zca_epsilon=.2, width_shift_range=0.2, height_shift_range=0, rotation_range=0, brightness_range=(0,1), shear_range=0, zoom_range=0.0)
# test_datagen = ImageDataGenerator(rescale=1./255) #don't add anything else to this one, only train datagen
# train_generator = train_datagen.flow_from_directory(train_dir, 
#                                                    target_size=(shape,shape),
#                                                    color_mode="rgb",
#                                                    batch_size=batch_size,
#                                                    class_mode="categorical",
#                                                    shuffle=True)
# test_generator = test_datagen.flow_from_directory(test_dir,
#                                                     target_size=(shape,shape),
#                                                     color_mode="rgb",
#                                                     batch_size=batch_size,
#                                                     class_mode="categorical"
#                                                     )
# history = x.fit_generator(train_generator,
#                     steps_per_epoch=nb_train_samples // batch_size+1,
#                     epochs=epochs,
#                     validation_data=test_generator,
#                     validation_steps=nb_test_samples // batch_size+1,
#                     workers=4,
#                     callbacks=callbacks
# )




#### Evaluate Model

In [31]:
from keras.models import load_model
model = load_model('best_model.h5', custom_objects={"NoisyAdam":noisy()})
model.evaluate_generator(test_generator, steps=nb_test_samples // batch_size+1, verbose=0)


[0.571805825191127, 0.8235829729657225]

# CNN Model #4

In [43]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import pandas as pd
from skimage import io
from PIL import Image
import functools
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from keras.metrics import categorical_accuracy
os.chdir("C:/Users/jerem.DESKTOP-GGM6Q2I/Documents/UNH Data Analytics/NN")

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

#### Neural Network Structure

In [44]:
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import MobileNet
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import BatchNormalization, Flatten, Dense, LeakyReLU, Dropout, GaussianNoise
from tensorflow import set_random_seed
from keras.optimizers import Adam
from keras_gradient_noise import add_gradient_noise
from keras.initializers import glorot_uniform, glorot_normal, he_uniform, he_normal
from keras.layers.convolutional import Conv2D, MaxPooling2D
set_random_seed(1724)
shape=64

batch_size = 128   
train_dir = "C:/Project/64/train" 
test_dir = "C:/Project/64/valid"  
nb_train_samples = sum([len(files) for r, d, files in os.walk(train_dir)])
nb_test_samples = sum([len(files) for r, d, files in os.walk(test_dir)])
epochs = 150




x = models.Sequential()

x.add(Conv2D(16, kernel_size = (3,3), strides = (1,1), padding = 'same', input_shape = (shape,shape,3)))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x.add(Flatten())
x.add(Dense(128, kernel_initializer="glorot_normal"))
x.add(LeakyReLU())
x.add(BatchNormalization())

x.add(Dense(38, activation='softmax')) 

#### Compiling Network

In [45]:
noisy = add_gradient_noise(Adam)
x.compile(optimizer=noisy(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])


#### Setup for Early Stopping

In [46]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

#### Train Model

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=False, vertical_flip=False, zca_whitening=True, zca_epsilon=.2, width_shift_range=0.2, height_shift_range=0, rotation_range=0, brightness_range=(0,1), shear_range=0, zoom_range=0.0)
test_datagen = ImageDataGenerator(rescale=1./255) #don't add anything else to this one, only train datagen
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                   target_size=(shape,shape),
                                                   color_mode="rgb",
                                                   batch_size=batch_size,
                                                   class_mode="categorical",
                                                   shuffle=True)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(shape,shape),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical"
                                                    )
history = x.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size+1,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=nb_test_samples // batch_size+1,
                    workers=4,
                    callbacks=callbacks
)




#### Evaluate Model

In [54]:
from keras.models import load_model
model = load_model('best_model.h5', custom_objects={"NoisyAdam":noisy()})
# model.evaluate_generator(test_generator, steps=nb_test_samples // batch_size+1, verbose=0)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(shape,shape),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical",
                                                    shuffle=False
                                                    )

predictions = model.predict_generator(test_generator, steps=nb_test_samples // batch_size+1, verbose=0)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())   

report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

Found 17572 images belonging to 38 classes.
                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       0.75      0.81      0.77       504
                                 Apple___Black_rot       0.68      0.90      0.77       497
                          Apple___Cedar_apple_rust       0.74      0.83      0.78       440
                                   Apple___healthy       0.74      0.78      0.76       502
                               Blueberry___healthy       0.75      0.83      0.79       454
          Cherry_(including_sour)___Powdery_mildew       0.87      0.80      0.83       421
                 Cherry_(including_sour)___healthy       0.80      0.95      0.87       456
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.79      0.71      0.74       410
                       Corn_(maize)___Common_rust_       0.91      0.96      0.93       477
               Corn_(maize)___North

# CNN Model #5

In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import pandas as pd
from skimage import io
from PIL import Image
import functools
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from keras.metrics import categorical_accuracy
os.chdir("C:/Users/jerem.DESKTOP-GGM6Q2I/Documents/UNH Data Analytics/NN")

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

Using TensorFlow backend.


#### Neural Network Structure

In [7]:
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import MobileNet
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.layers import BatchNormalization, Flatten, Dense, LeakyReLU, Dropout, GaussianNoise
from tensorflow import set_random_seed
from keras.optimizers import Adam
from keras_gradient_noise import add_gradient_noise
from keras.initializers import glorot_uniform, glorot_normal, he_uniform, he_normal
from keras.layers.convolutional import Conv2D, MaxPooling2D
set_random_seed(1724)
shape=64

batch_size = 128   
train_dir = "C:/Project/64/train" 
test_dir = "C:/Project/64/valid"  
nb_train_samples = sum([len(files) for r, d, files in os.walk(train_dir)])
nb_test_samples = sum([len(files) for r, d, files in os.walk(test_dir)])
epochs = 150



# base_model=VGG16(weights="imagenet", include_top=False, input_shape=(shape,shape,3))
# base_model.summary()
# base_model = InceptionV3(weights="imagenet", include_top=False)
# base_model.summary()


x = models.Sequential()

x.add(Conv2D(16, kernel_size = (3,3), strides = (1,1), padding = 'same', input_shape = (shape,shape,3)))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(16, kernel_size = (3,3), strides = (1,1), padding = 'same', input_shape = (shape,shape,3)))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
x.add(LeakyReLU())
x.add(BatchNormalization())
x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x.add(Flatten())
x.add(Dense(1024, kernel_initializer="glorot_normal"))
x.add(LeakyReLU())
x.add(BatchNormalization())


x.add(Dense(38, activation='softmax')) 


#### Compiling Network

In [67]:
noisy = add_gradient_noise(Adam)
x.compile(optimizer=noisy(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])


#### Setup for Early Stopping

In [68]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

#### Train Model

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=False, vertical_flip=False, zca_whitening=True, zca_epsilon=.2, width_shift_range=0.2, height_shift_range=0, rotation_range=0, brightness_range=(0,1), shear_range=0, zoom_range=0.0)
test_datagen = ImageDataGenerator(rescale=1./255) #don't add anything else to this one, only train datagen
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                   target_size=(shape,shape),
                                                   color_mode="rgb",
                                                   batch_size=batch_size,
                                                   class_mode="categorical",
                                                   shuffle=True)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(shape,shape),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical"
                                                    )
history = x.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size+1,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=nb_test_samples // batch_size+1,
                    workers=4,
                    callbacks=callbacks
)




#### Evaluate Model

In [70]:
from keras.models import load_model
model = load_model('best_model.h5', custom_objects={"NoisyAdam":noisy()})
# model.evaluate_generator(test_generator, steps=nb_test_samples // batch_size+1, verbose=0)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(shape,shape),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical",
                                                    shuffle=False
                                                    )

predictions = model.predict_generator(test_generator, steps=nb_test_samples // batch_size+1, verbose=0)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())   

report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

Found 17572 images belonging to 38 classes.
                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       0.84      0.79      0.81       504
                                 Apple___Black_rot       0.88      0.85      0.87       497
                          Apple___Cedar_apple_rust       0.91      0.84      0.87       440
                                   Apple___healthy       0.90      0.79      0.84       502
                               Blueberry___healthy       0.94      0.78      0.85       454
          Cherry_(including_sour)___Powdery_mildew       0.86      0.95      0.90       421
                 Cherry_(including_sour)___healthy       0.89      0.97      0.93       456
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.90      0.77      0.83       410
                       Corn_(maize)___Common_rust_       0.94      0.99      0.96       477
               Corn_(maize)___North

# CNN Model #6

In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import pandas as pd
from skimage import io
from PIL import Image
import functools
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from keras.metrics import categorical_accuracy
os.chdir("C:/Users/jerem.DESKTOP-GGM6Q2I/Documents/UNH Data Analytics/NN")

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

Using TensorFlow backend.


#### Neural Network Structure

In [5]:
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import MobileNet
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.layers import BatchNormalization, Flatten, Dense, LeakyReLU, Dropout, GaussianNoise
from tensorflow import set_random_seed
from keras.optimizers import Adam
from keras_gradient_noise import add_gradient_noise
from keras.initializers import glorot_uniform, glorot_normal, he_uniform, he_normal
from keras.layers.convolutional import Conv2D, MaxPooling2D
set_random_seed(1724)
shape=128

batch_size = 128   
train_dir = "C:/Project/128/train" 
test_dir = "C:/Project/128/valid"  
nb_train_samples = sum([len(files) for r, d, files in os.walk(train_dir)])
nb_test_samples = sum([len(files) for r, d, files in os.walk(test_dir)])
epochs = 150



# base_model=VGG16(weights="imagenet", include_top=False, input_shape=(shape,shape,3))
# base_model.summary()
# base_model = InceptionV3(weights="imagenet", include_top=False)
# base_model.summary()


base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(shape,shape,3))
x = base_model.output

# x.add(Conv2D(16, kernel_size = (3,3), strides = (1,1), padding = 'same', input_shape = (shape,shape,3)))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(16, kernel_size = (3,3), strides = (1,1), padding = 'same', input_shape = (shape,shape,3)))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

# x.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

# x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

# x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

# x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(Conv2D(256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
# x.add(LeakyReLU())
# x.add(BatchNormalization())
# x.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))

x = (Flatten())(x)
x = (Dense(1024, kernel_initializer="glorot_normal"))(x)
x = (LeakyReLU())(x)
x = (BatchNormalization())(x)


pred = (Dense(38, activation='softmax'))(x)
model = Model(inputs=base_model.input, outputs=pred)


#### Compiling Network

In [7]:
noisy = add_gradient_noise(Adam)
model.compile(optimizer=noisy(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])


#### Setup for Early Stopping

In [8]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

#### Train Model

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=False, vertical_flip=False, zca_whitening=True, zca_epsilon=.2, width_shift_range=0.2, height_shift_range=0, rotation_range=0, brightness_range=(0,1), shear_range=0, zoom_range=0.0)
test_datagen = ImageDataGenerator(rescale=1./255) #don't add anything else to this one, only train datagen
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                   target_size=(shape,shape),
                                                   color_mode="rgb",
                                                   batch_size=batch_size,
                                                   class_mode="categorical",
                                                   shuffle=True)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(shape,shape),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical"
                                                    )
history = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size+1,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=nb_test_samples // batch_size+1,
                    workers=4,
                    callbacks=callbacks
)




#### Evaluate Model

In [70]:
from keras.models import load_model
model = load_model('best_model.h5', custom_objects={"NoisyAdam":noisy()})
# model.evaluate_generator(test_generator, steps=nb_test_samples // batch_size+1, verbose=0)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(shape,shape),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode="categorical",
                                                    shuffle=False
                                                    )

predictions = model.predict_generator(test_generator, steps=nb_test_samples // batch_size+1, verbose=0)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())   

report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

Found 17572 images belonging to 38 classes.
                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       0.84      0.79      0.81       504
                                 Apple___Black_rot       0.88      0.85      0.87       497
                          Apple___Cedar_apple_rust       0.91      0.84      0.87       440
                                   Apple___healthy       0.90      0.79      0.84       502
                               Blueberry___healthy       0.94      0.78      0.85       454
          Cherry_(including_sour)___Powdery_mildew       0.86      0.95      0.90       421
                 Cherry_(including_sour)___healthy       0.89      0.97      0.93       456
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.90      0.77      0.83       410
                       Corn_(maize)___Common_rust_       0.94      0.99      0.96       477
               Corn_(maize)___North